In [ ]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from PIL import Image
from urllib.request import urlopen

In [ ]:
# Data
trainFolder = "C://users/Administrator/Desktop/horse-or-human/train"
testFolder = "C://users/Administrator/Desktop/horse-or-human/validation"

In [ ]:
listLabels = labels = ['Horse','Human']

In [ ]:
# Image generator
imageGenerator = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip=True,
    zoom_range=0.15,
    zca_whitening=True,
    fill_mode='nearest')

# training images
trainImages = imageGenerator.flow_from_directory(
    trainFolder,
    target_size=(48,48),
    class_mode='categorical'
)



# test Images
testImages = ImageDataGenerator(rescale=1./255)
testImages = imageGenerator.flow_from_directory(
        testFolder,
        target_size=(48, 48),
        class_mode='categorical')

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=(48,48,3)),
        layers.Flatten(),
        layers.BatchNormalization(),
        layers.Dense(128, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(32, activation="relu"),
        layers.Dense(2, activation="softmax"),
    ]
)

model.build((48, 48))
model.summary()
#opt = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: 
            print('')
            print("Epoch",epoch,'completed', end='')

            
qtyEpochs = 50
history = model.fit(trainImages, validation_data=testImages,batch_size=50, shuffle=True, epochs=qtyEpochs, callbacks=[early_stop,PrintDot()], verbose=1)

In [ ]:
model_eval = model.evaluate_generator (
    testImages,
    verbose=0
)

print('Model accuracy:',model_eval[1])
print('Model loss:',model_eval[0])

In [ ]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # To use local CPU instead of local GPU
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as implt
from PIL import Image 
import time,datetime,keras,cv2,shutil,keras_preprocessing,requests
import tensorflow as tf
from keras import layers
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D 
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from io import BytesIO
import seaborn as sns
%load_ext tensorboard

In [ ]:
import urllib.request
from PIL import Image
image = "https://storage.googleapis.com/filesbucketep/horse3-498.png"
test01 = Image.open(urllib.request.urlopen(image))
test01

In [ ]:
test01 = test01.convert('RGB')
test01 = test01.resize((48,48))
test01_arr = np.asarray(test01) / 255
pred = model.predict(np.array([test01_arr]))
print("Pred:",labels[pred.argmax(axis=-1)[0]])